In [1]:
import os
import openai
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']
from GraceAgent.LLMPlanner import GracePlanner
from Utils.PromptTemplateBuilder import PromptTemplateBuilder
from langchain_core.prompts.chat import SystemMessagePromptTemplate,ChatPromptTemplate

In [2]:
from Tools import *
from Tools.PythonTool import xiaoyaAnalyser

In [3]:
tools = [
    document_generation_tool,
    directory_inspection_tool,
    CSV_inspection_tool,
    finish_placeholder,
    xiaoyaAnalyser(
        prompt_path="./prompts/Tools",
        info_path="./Tools/xiaoya_info"
    ).as_tool()
]

In [4]:
model = ChatOpenAI(model="gpt-4",temperature=0)
planner = GracePlanner(model, tools, stop=['<END_OF_PLAN>'])

In [ ]:
from GraceAgent import GraceExecutor

executor = GraceExecutor(
        llm=model,
        prompts_path="./prompts/main",
        tools=tools,
        work_dir='datasets',
        main_prompt_file='executor.json',
        final_prompt_file='final_step.json',
        max_thought_steps=10,
    )

In [ ]:
executor.run({"previous_steps":"","current_step":"读取数据集以获取男女信息。"})

In [ ]:
prompt = PromptTemplateBuilder(prompt_path="./prompts/executor", prompt_file="executor.json").build(
    tools=tools
).partial(
            work_dir='./datasets',
    current_step="",
format_instructions = "", 
input="", 
previous_step="", 
short_term_memory=""
        )

In [ ]:
print(prompt.format())

In [5]:
tools = [
    document_generation_tool,
    directory_inspection_tool,
    CSV_inspection_tool,
    finish_placeholder,
    xiaoyaAnalyser(
        prompt_path="./prompts/Tools",
        info_path="./Tools/xiaoya_info"
    ).as_tool()
]
model = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0,model_kwargs={"seed": 42})
planner = GracePlanner(model, tools, stop=['<END_OF_PLAN>'])

In [8]:
tools

[StructuredTool(name='GenerateDocument', description='GenerateDocument(query: str) - 根据需求描述生成一篇正式文档', args_schema=<class 'pydantic.v1.main.GenerateDocumentSchema'>, func=<function write at 0x12b134af0>),
 StructuredTool(name='ListDirectory', description='ListDirectory(path: str) -> str - 探查文件夹的内容和结构，展示它的文件名和文件夹名', args_schema=<class 'pydantic.v1.main.ListDirectorySchema'>, func=<function list_files_in_directory at 0x12b134d30>),
 StructuredTool(name='InspectCSV', description='InspectCSV(filename: str, n: int = 3) -> str - 探查表格文件的内容和结构，展示它的列名和前n行，n默认为3', args_schema=<class 'pydantic.v1.main.InspectCSVSchema'>, func=<function get_first_n_rows at 0x12b1351b0>),
 StructuredTool(name='FINISH', description='FINISH() - 用于表示任务完成的占位符：FINISH', args_schema=<class 'pydantic.v1.main.FINISHSchema'>, func=<function <lambda> at 0x12b135240>),
 StructuredTool(name='AnalyseCSV', description='AnalyseCSV(inputs) - 通过程序脚本分析一个结构化文件（例如CSV文件）的内容。输入必须是以字典的形式进行指明,其中包含三个键： type、files和content。\n            type:只

In [9]:
[(tool.name,tool.description) for tool in tools]

[('GenerateDocument', 'GenerateDocument(query: str) - 根据需求描述生成一篇正式文档'),
 ('ListDirectory',
  'ListDirectory(path: str) -> str - 探查文件夹的内容和结构，展示它的文件名和文件夹名'),
 ('InspectCSV',
  'InspectCSV(filename: str, n: int = 3) -> str - 探查表格文件的内容和结构，展示它的列名和前n行，n默认为3'),
 ('FINISH', 'FINISH() - 用于表示任务完成的占位符：FINISH'),
 ('AnalyseCSV',
  'AnalyseCSV(inputs) - 通过程序脚本分析一个结构化文件（例如CSV文件）的内容。输入必须是以字典的形式进行指明,其中包含三个键： type、files和content。\n            type:只能是(数据预处理、数据分析、数据可视化、其他)其中一项\n            files:文件的完整路径,可以是多个文件\n            content:尽可能完整的阐述当前分析阶段、具体分析方式和分析依据，阈值常量等。\n            如果输入信息不完整，你可以拒绝回答。')]

In [6]:
print(planner.plan({'input':"请帮我处理一下数据集",'previous_steps':""}))

steps=[Step(value='使用ListDirectory动作，获取"./datasets"文件夹中的文件列表。'), Step(value='对于"./datasets"文件夹中的每个CSV文件，使用xiaoya包的一键数据预处理功能进行处理。'), Step(value='完成数据预处理后，任务结束。\n<END_OF_PLAN>')]


In [1]:
from Tools.Interpreter import Interpreter

In [2]:
interpreter = Interpreter()

In [4]:
code1 = '''
x = 10
y = 20
print(x + y)
'''

In [4]:
code2 = '''
z = x * y
print(z)
'''

In [5]:
interpreter.execute(code1)

30


In [6]:
interpreter.execute(code2)

200


In [1]:
from Tools.Interpreter import Interpreter

In [2]:
interpreter = Interpreter()

In [5]:
ans = interpreter.execute(code1)

30


In [6]:
ans

'30\n'

In [13]:
tool.run(code1)

30


In [17]:
ans = tool.run(code2)

200


In [18]:
ans

In [15]:
tas = [("halahl","heiheihiehei"),("1234","ajkajk")]

In [16]:
str(tas)

"[('halahl', 'heiheihiehei'), ('1234', 'ajkajk')]"